In [ ]:
import pandas as pd
import numpy as np
import quantreo.target_engineering as te

# CONFIG
HOLDING_PERIOD_HOURS = 168  # 7 days * 24 hours = 168 hours
TAKE_PROFIT = 0.015  # 1.5% profit target (adjust based on volatility)
STOP_LOSS = -0.015  # -1.5% stop loss
BUY_SIDE = True  # Long positions only

# LOAD CACHED DATA
processed_data_cache = pd.read_pickle('data_cache/processed_data.pkl')
bitcoin_price_and_features = processed_data_cache['bitcoin_price_and_features']

# Start with normalised features
labelling_df = processed_data_cache['features_norm'].copy()

# Add OHLC data (NOT normalised) for Quantreo
labelling_df['open'] = bitcoin_price_and_features[('Open', 'BTC-USD')]
labelling_df['high'] = bitcoin_price_and_features[('High', 'BTC-USD')]
labelling_df['low'] = bitcoin_price_and_features[('Low', 'BTC-USD')]
labelling_df['close'] = bitcoin_price_and_features[('Close', 'BTC-USD')]

# Ensure DatetimeIndex named 'time'
labelling_df.index.name = 'time'

# Simulate high_time and low_time (daily data limitation)
labelling_df['high_time'] = labelling_df.index + pd.Timedelta(hours=10)
labelling_df['low_time'] = labelling_df.index + pd.Timedelta(hours=14)

# APPLY QUANTREO TRIPLE BARRIER LABELLING
print("Applying Quantreo triple barrier labelling...")
labelling_df['Label_7day'] = te.directional.triple_barrier_labeling(
    labelling_df,
    168,
    open_col='open',
    high_col='high',
    low_col='low',
    high_time_col='high_time',
    low_time_col='low_time',
    tp=TAKE_PROFIT,
    sl=STOP_LOSS,
    buy=BUY_SIDE
)

# VALIDATION
print("=" * 50)
print("QUANTREO TRIPLE BARRIER RESULTS")
print("=" * 50)
print(f"Total rows: {len(labelling_df)}")
print(f"Valid labels: {labelling_df['Label_7day'].notna().sum()}")
print(f"\nLabel distribution:")
print(labelling_df['Label_7day'].value_counts().sort_index())
print(f"\nLabel percentages:")
print(labelling_df['Label_7day'].value_counts(normalize=True).sort_index())